In [19]:
import cx_Oracle
import pandas as pd
import os
import sys

os.chdir(r'D:\10academy\Bank-reviews-analysis_W2\Bank-reviews-analysis')
sys.path.append(os.getcwd()) 
# Connect to the Oracle database

hostname = 'localhost'
port = 1521
sid = 'localcr'
username = 'bank_reviews'
password = 'bankreviews'
dsn = cx_Oracle.makedsn(hostname, port, sid=sid)

# Connect to the Oracle database
connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)

# Create a cursor
cursor = connection.cursor()

# Paths to the processed bank data CSV files
banks = {
    "CBE": "notebooks/data/processed_reviews_CBE.csv",
    "BOA": "notebooks/data/processed_reviews_BOA.csv",
    "Dashen": "notebooks/data/processed_reviews_Dashen.csv"
}

# Load the processed bank data into DataFrames
combined_reviews = pd.DataFrame()

for bank, path in banks.items():
    # Load each bank's processed reviews
    bank_df = pd.read_csv(path)
   
    combined_reviews = pd.concat([combined_reviews, bank_df], ignore_index=True)
# Convert the date column to datetime format
combined_reviews['date'] = pd.to_datetime(combined_reviews['date'], errors='coerce')

# Check for any NaT values after conversion
if combined_reviews['date'].isnull().any():
    print("Warning: Some date values could not be converted!")

if 'processed_review' not in combined_reviews.columns or combined_reviews['processed_review'].isnull().any():
    combined_reviews['processed_review'] = combined_reviews['processed_review'].fillna('')
# Ensure it's a string type
combined_reviews['processed_review'] = combined_reviews['processed_review'].astype(str)

for index, row in combined_reviews.iterrows():
     cursor.execute("""
        INSERT INTO reviews (review_text, rating, review_date, bank_name, source, processed_review, sentiment_score, sentiment)
        VALUES (:1, :2, :3, :4, :5, :6, :7, :8)
    """, (row['review_text'], row['rating'], row['date'], row['bank_name'], row['source'], row['processed_review'], row['sentiment_score'], row['sentiment']))

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("processed Data inserted successfully to reviews table!")

processed Data inserted successfully to reviews table!
